<a href="https://colab.research.google.com/github/ferdouszislam/pytorch-practice/blob/main/pytorch_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import torch
import numpy as np

In [18]:
if torch.cuda.is_available():
  print('GPU, yay!')
else:
  print('CPU :(')

CPU :(


# Tutorial 02 - Tensor Basics

In [19]:
x = torch.rand(3,3)
y = torch.rand(3,3)

In [20]:
print(x, '\n', y, '\n')

tensor([[0.1024, 0.3566, 0.2601],
        [0.3519, 0.7763, 0.7665],
        [0.4996, 0.3259, 0.4082]]) 
 tensor([[0.2476, 0.7871, 0.6586],
        [0.6370, 0.3238, 0.3171],
        [0.6824, 0.9298, 0.0228]]) 



In [21]:
z = torch.add(x, y)
z

tensor([[0.3499, 1.1437, 0.9188],
        [0.9889, 1.1000, 1.0836],
        [1.1820, 1.2557, 0.4310]])

In [22]:
z = torch.mul(x, y)
z 

tensor([[0.0253, 0.2807, 0.1713],
        [0.2242, 0.2513, 0.2430],
        [0.3409, 0.3030, 0.0093]])

In [23]:
z.add_(y) # same as z+=y

tensor([[0.2729, 1.0678, 0.8300],
        [0.8612, 0.5751, 0.5601],
        [1.0233, 1.2328, 0.0321]])

In [24]:
z[:, 2] # get all rows at column 2 (0 based indexing)

tensor([0.8300, 0.5601, 0.0321])

In [25]:
z[1,2].item() # get single element value

0.5601442456245422

In [26]:
'''
input- a pytorch tensor variable 
returns- multiplication of the input tensor's dimensions 
'''
def get_flat_shape(tensor):
  dims=list(tensor.size())
  flat_dim = 1
  for dim in dims:
    flat_dim*=dim
  return flat_dim

In [27]:
get_flat_shape(z)

9

In [28]:
flat_z = z.view(get_flat_shape(z)) # resizing a tensor
flat_z

tensor([0.2729, 1.0678, 0.8300, 0.8612, 0.5751, 0.5601, 1.0233, 1.2328, 0.0321])

In [29]:
# tensor to numpy array conversion
np_z = flat_z.clone().numpy() # using '.clone()' is a MUST
np_z

array([0.27290922, 1.0678104 , 0.82997155, 0.86116344, 0.5750936 ,
       0.56014425, 1.0232682 , 1.2327998 , 0.03206977], dtype=float32)

In [30]:
# numpy array to tensor conversion
flat_z = torch.from_numpy(np_z.copy()) # using '.copy()' is a MUST
flat_z

tensor([0.2729, 1.0678, 0.8300, 0.8612, 0.5751, 0.5601, 1.0233, 1.2328, 0.0321])

  
  **Tensors can be kept into GPU but numpy arrays have to remain on CPU. GPUs are generally faster.**  


In [31]:
device = False
if torch.cuda.is_available():
  device = torch.device("cuda")

In [32]:
# all operations on tensors to be done in GPU
# x = torch.rand(2, 2).to(device)
# y = torch.rand(2, 2).to(device)
# z=x+y

# print(x, '\n', y, '\n', z)

In [33]:
# numpy arrays MUST be on cpu
z = z.to('cpu')
np_z = z.clone().numpy()
np_z

array([[0.27290922, 1.0678104 , 0.82997155],
       [0.86116344, 0.5750936 , 0.56014425],
       [1.0232682 , 1.2327998 , 0.03206977]], dtype=float32)

# Tutorial 03 - Gradient Calculation with Autograd

In [34]:
x = torch.randn(3, requires_grad=True)
x

tensor([-0.6741,  1.2601, -0.7607], requires_grad=True)

In [35]:
y=x+2
print(y)
z=y*y*2
print(z)
z = z.mean()
print(z) 

tensor([1.3259, 3.2601, 1.2393], grad_fn=<AddBackward0>)
tensor([ 3.5162, 21.2568,  3.0718], grad_fn=<MulBackward0>)
tensor(9.2816, grad_fn=<MeanBackward0>)


In [36]:
# calculating dz/dx for each element of x tensor (in this case- x1,x2,x3)
#  N.B- All but the last call to backward should have the retain_graph=True option
z.backward(retain_graph=True)
print(x.grad)

tensor([1.7679, 4.3468, 1.6524])


In [37]:
# prevent gradient tracking 
# (might be needed when updating weights during training)

x = torch.randn(3, requires_grad=True)
print(x)

# way 1
print('way 1')
y=x
y.requires_grad_(False)
print(x)

# way 2
print('way 2')
y=x.detach()
print(y)

# way 3
print('way 3')
y=x+2
print(x, '\n', y)
with torch.no_grad():
  y=x+2
  print(x, '\n', y)

tensor([-1.3022, -1.7763, -1.6780], requires_grad=True)
way 1
tensor([-1.3022, -1.7763, -1.6780])
way 2
tensor([-1.3022, -1.7763, -1.6780])
way 3
tensor([-1.3022, -1.7763, -1.6780]) 
 tensor([0.6978, 0.2237, 0.3220])
tensor([-1.3022, -1.7763, -1.6780]) 
 tensor([0.6978, 0.2237, 0.3220])


In [38]:
# dummy training example with some weights

weights = torch.ones(4, requires_grad=True)

for epoch in range(3):
  model_output = (weights*3).sum() # loss function... probably
  
  model_output.backward()
  print(weights.grad)

  # before next iteration or optimization step MUST empty the gradient
  weights.grad.zero_()

tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])
tensor([3., 3., 3., 3.])


# Tutorial 04 - Back Propagation

In [39]:
# example backpropagation for a single instance

x = torch.tensor(1.0) # input
y = torch.tensor(2.0) # actual output

w = torch.tensor(1.0, requires_grad=True) # weight i.e, learnable parameter

# forward pass
y_hat = w*x # y_hat is the prediction using linear model = w*x 
loss = (y_hat - y)**2 # loss function = squared error (generally this would be MSE)

print(loss)

# backward pass
loss.backward()
print(w.grad)

# now update weight using the gradient 
# and do forward and backward pass again

tensor(1., grad_fn=<PowBackward0>)
tensor(-2.)


# Tutorial 05 - Gradient Descent with Autograd & Backpropagation



In [40]:
# implementing linear regression from scratch with dummy data

# f = w*x, for the example model below w = 2 fits the ouput completely 
X = np.array([1,2,3,4], dtype=np.float32) # input
Y = np.array([2,4,6,8], dtype=np.float32) # actual output

# randomely initializing weight
w = 0.0

# model prediction
def forward(x):
  return w*x

# loss function, MSE
def loss(y, y_pred):
  return ((y_pred-y)**2).mean()

# gradient
# here, loss, J = 1/N * (w*x-y)^2 [because y_pred = w*x]
# therefore, dJ/dw = 1/N*2*x*(w*x-y)
def gradient(x, y, y_pred):
  return np.dot(2*x, y_pred-y).mean()

print(f'Prediction before training for x=5 : {forward(5):.3f}')

# Training
learning_rate = 0.01
n_iters = 5

print('\n[Training started...]\n')
for epoch in range(n_iters):
  # prediction, forward pass
  y_pred = forward(X)

  # loss
  J = loss(Y, y_pred)

  # gradient
  dJ_dw = gradient(X, Y, y_pred)

  # update weights
  w = w - learning_rate* dJ_dw 

  # print everytime
  if epoch%1==0:
    print(f'epoch {epoch+1}: w = {w:.3f}, loss = {J:.8f}')
print('\n[Training finished...]\n')

print(f'Prediction after training for x=5 : {forward(5):.3f}')

Prediction before training for x=5 : 0.000

[Training started...]

epoch 1: w = 1.200, loss = 30.00000000
epoch 2: w = 1.680, loss = 4.79999924
epoch 3: w = 1.872, loss = 0.76800019
epoch 4: w = 1.949, loss = 0.12288000
epoch 5: w = 1.980, loss = 0.01966083

[Training finished...]

Prediction after training for x=5 : 9.898


### Now let's do the same using Pytorch

In [41]:
# implementing linear regression from scratch with dummy data

# f = w*x, for the example model below w = 2 fits the ouput completely 
X = torch.tensor([1,2,3,4], dtype=torch.float32) # input
Y = torch.tensor([2,4,6,8], dtype=torch.float32) # actual output

# randomely initializing weight
w = torch.tensor(0.0, dtype=torch.float32, requires_grad=True)

# model prediction
def forward(x):
  return w*x

# loss function, MSE
def loss(y, y_pred):
  return ((y_pred-y)**2).mean()

# gradient
# here, loss, J = 1/N * (w*x-y)^2 [because y_pred = w*x]
# therefore, dJ/dw = 1/N*2*x*(w*x-y)
# def gradient(x, y, y_pred):
#   return np.dot(2*x, y_pred-y).mean()

print(f'Prediction before training for x=5 : {forward(5):.3f}')

# Training
learning_rate = 0.01
n_iters = 20

print('\n[Training started...]\n')
for epoch in range(n_iters):
  # prediction i.e forward pass
  y_pred = forward(X)

  # loss
  J = loss(Y, y_pred)

  # calculate gradient i.e backward pass
  #dJ_dw = gradient(X, Y, y_pred)
  J.backward() # dJ/dw

  # update weights
  #w = w - learning_rate* dJ_dw 
  w.data = w.data - learning_rate * w.grad
  # alternately we can do this,
  # with torch.no_grad(): 
  #   # update to weight should not be tracked for calculating gradient
  #   w -= learning_rate*w.grad

  w.grad.zero_() # clear the gradients 

  if epoch%2==1:
    print(f'epoch {epoch+1}: w = {w:.3f}, loss = {J:.8f}')
print('\n[Training finished...]\n')

print(f'Prediction after training for x=5 : {forward(5):.3f}')

Prediction before training for x=5 : 0.000

[Training started...]

epoch 2: w = 0.555, loss = 21.67499924
epoch 4: w = 0.956, loss = 11.31448650
epoch 6: w = 1.246, loss = 5.90623236
epoch 8: w = 1.455, loss = 3.08308983
epoch 10: w = 1.606, loss = 1.60939169
epoch 12: w = 1.716, loss = 0.84011245
epoch 14: w = 1.794, loss = 0.43854395
epoch 16: w = 1.851, loss = 0.22892261
epoch 18: w = 1.893, loss = 0.11949898
epoch 20: w = 1.922, loss = 0.06237914

[Training finished...]

Prediction after training for x=5 : 9.612
